In [1]:
import numpy as np
import time
import cv2
import os

In [2]:
### numpy

In [3]:
const255np = 255.0
const128np = 128.0

In [4]:
### Nested

In [5]:
def darken_blend_8_np(base, active):
  return np.where(np.greater(base, active), active, base)

In [6]:
def color_burn_8_np(base, active):
  return np.where(np.equal(active, 0), const255np, const255np - (const255np - base) / active)

In [7]:
def lighten_blend_8_np(base, active):
  return np.where(np.less(base, active), active, base)

In [8]:
def color_dodge_8_np(base, active):
  return np.where(np.equal(active, const255np), const255np, base / (const255np - active))

In [9]:
def overlay_blend_8_np(base, active):
  return np.where(np.greater_equal(base, const128np), 2 * base + base - 2 * base * base / const255np - const128np, 2 * base * base / const128np)

In [10]:
def multiply_blend_8_np(base, active):
  return base * active / const255np

In [11]:
def linear_burn_8_np(base, active):
  return base + active - const255np

In [12]:
def screen_blend_8_np(base, active):
  return base + active - base * active / const255np

In [13]:
def linear_dodge_8_np(base, active):
  return base + active

In [14]:
### Single

In [15]:
def normal_blend_f_np(base, active, opacity):
  return opacity * active + (1-opacity)*base

In [16]:
def normal_blend_8_np(base, active, opacity):
  return opacity * active + (const255np - opacity) * base

In [17]:
# Runner

In [18]:
size = 512
np.random.seed(1)

list1 = np.random.rand(size).astype('float32')
list2 = np.random.rand(size).astype('float32')
list3 = np.random.rand(size)

int1 = np.random.rand(1).astype('float32')

In [19]:
folder = "./data/"

img_files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

bases = []
actives = []

for _file in img_files:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE)
    rnd = np.random.rand(*img.shape).astype('float32')

    bases.append(img)
    actives.append(rnd)

In [20]:
def runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = bases[i]
        a = actives[i]
        start_time = time.perf_counter()
        f(b, a)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def mat_timer(bases, actives, f):
    runs = 5
    times = []
    for _ in range(runs):
        times.append(runner(bases, actives, f))
    times = np.array(times)
    print(f"{np.average(times)}ms +/- {np.std(times)}ms")

In [21]:
mat_timer(bases, actives, darken_blend_8_np)

1617.4099280498922ms +/- 44.254167836991954ms


In [22]:
mat_timer(bases, actives, color_burn_8_np) 

4302.086239261553ms +/- 23.140287768617558ms


In [23]:
mat_timer(bases, actives, lighten_blend_8_np)

1622.2720935475081ms +/- 4.8468646682959395ms


In [24]:
mat_timer(bases, actives, color_dodge_8_np)

2114.2432576045394ms +/- 7.342991852991134ms


In [25]:
mat_timer(bases, actives, overlay_blend_8_np)

7191.760542243719ms +/- 8.515459911827737ms


In [26]:
mat_timer(bases, actives, multiply_blend_8_np)

985.2184700779617ms +/- 6.285905208942349ms


In [27]:
mat_timer(bases, actives, linear_burn_8_np)

933.6093899793923ms +/- 39.7617391549321ms


In [28]:
mat_timer(bases, actives, screen_blend_8_np)

1859.525149082765ms +/- 7.8551838415631785ms


In [29]:
mat_timer(bases, actives, linear_dodge_8_np)

731.9464105181396ms +/- 46.5453469155274ms


In [30]:
%timeit list3 = normal_blend_f_np(list1, list2, int1)

1.9 µs ± 2.39 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [31]:
%timeit list3 = normal_blend_8_np(list1, list2, int1)

1.92 µs ± 3.49 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
